In [3]:
import argparse
import math
import builtins
import datetime
import gradio
import os
import torch
import numpy as np
import functools
import trimesh
import copy
from scipy.spatial.transform import Rotation

from dust3r.inference import inference
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images, rgb
from dust3r.utils.device import to_numpy
from dust3r.viz import add_scene_cam, CAM_COLORS, OPENGL, pts3d_to_trimesh, cat_meshes
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import matplotlib.pyplot as pl

from dust3r.demo import get_reconstructed_scene, get_3D_model_from_scene
from dust3r.model import AsymmetricCroCo3DStereo

In [2]:
outdir = "output"
device = "cuda"
silent = False
image_size = 512
filelist = "./images_in"
schedule = "linear" # or "cosine"
niter = 300 # number of iters
min_conf_thr = 3
as_pointcloud = True
mask_sky = False
clean_depth = True
transparent_cams = False
cam_size = 0.05
scenegraph_type = "complete"
winsize = 1
refid = 0

model = AsymmetricCroCo3DStereo.from_pretrained("./docker/files/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth").to(device)


... loading model from ./docker/files/checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [4]:
scene, pts3d, rgbimg, cams2world, confs = get_reconstructed_scene(outdir, model, device, silent, image_size, filelist, schedule, niter, min_conf_thr,
                            as_pointcloud, mask_sky, clean_depth, transparent_cams, cam_size,
                            scenegraph_type, winsize, refid)


>> Loading images from ./images_in
 - adding Screenshot 2024-09-25 232957.png with resolution 183x357 --> 256x512
 (Found 1 images)
>> Inference with model on 2 image pairs


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.09it/s]


  - conf=6.43 for edge 0-1
  - conf=6.43 for edge 1-0


In [14]:
mask = to_numpy(scene.get_masks())

pts3d = to_numpy(pts3d)


pts = np.concatenate([p[m] for p, m in zip(pts3d, mask)])
col = np.concatenate([p[m] for p, m in zip(rgbimg, mask)])
conf = np.concatenate([p[m] for p, m in zip(confs, mask)])

np.savetxt("./output/out.csv",np.hstack((pts, col, conf)), delimiter=",")


LICENSE		     docker	       requirements_optional.txt
NOTICE		     dust3r	       simple_interface.ipynb
README.md	     dust3r_visloc     simple_interface.py
assets		     images_in	       train.py
croco		     old_README.md     visloc.py
datasets_preprocess  output
demo.py		     requirements.txt
